In [1]:
import numpy as np

In [2]:
ACTION_SPACE = ('U', 'D', 'L', 'R')

In [3]:
class Grid:
    def __init__(self, rows, columns, start):
        self.rows = rows
        self.columns = columns
        self.i = start[0]
        self.j = start[1]

    def set_rewards_actions(self, rewards, actions):
        self.rewards = rewards
        self.actions = actions
    
    def get_state(self):
        return (self.i, self.j)

    def set_state(self, s):
        self.i = s[0]
        self.j = s[1]

    def get_next_state(self, s, a):
        i, j = s[0], s[1]
        if a in self.actions[(i, j)]:
            if a == 'U':
                i -= 1
            elif a == 'D':
                i += 1
            elif a == 'L':
                j -= 1
            elif a == 'R':
                j += 1
        return i, j
    
    def move(self, a):
        if a in self.actions[(self.i, self.j)]:
            if a == 'U':
                self.i -= 1
            elif a == 'D':
                self.i += 1
            elif a == 'L':
                self.j -= 1
            elif a == 'R':
                self.j += 1
        return self.rewards.get((self.i, self.j), 0)
    
    def undo_move(self, a):
        if a == 'U':
            self.i += 1
        elif a == 'D':
            self.i -= 1
        elif a == 'L':
            self.j += 1
        elif a == 'R':
            self.j -= 1
        assert (self.get_state() in self.all_states())
    
    def is_terminal(self, s):
        return s not in self.actions

    def game_over(self):
        return (self.i, self.j) not in self.actions

    def all_states(self):
        return set(self.actions.keys() | self.rewards.keys())

In [4]:
def grid_standard():
    g = Grid(3, 4, (2, 0))
    rewards = {
        (0,3): 1,
        (1,3): -1
    }
    
    actions = {
        (0, 0): ('D', 'R'),
        (0, 1): ('L', 'R'),
        (0, 2): ('L', 'D', 'R'),
        (1, 0): ('U', 'D'),
        (1, 2): ('U', 'D', 'R'),
        (2, 0): ('U', 'R'),
        (2, 1): ('L', 'R'),
        (2, 2): ('U', 'L', 'R'),
        (2, 3): ('U', 'L')
    }

    g.set_rewards_actions(rewards, actions)

    return g

In [5]:
def print_values(V, g):
    for i in range(g.rows):
        print("-------------------------")
        for j in range(g.columns):
            v = V.get((i, j), 0)
            if v >= 0:
                print(" %.2f|" % v, end="")
            else:
                print("%.2f|" % v, end="")
        print("")

def print_policy(P, g):
    for i in range(g.rows):
        print("-------------------------")
        for j in range(g.columns):
            a = P.get((i, j), ' ')
            print(" %s |" % a, end="")
        print("")

In [6]:
def t_probs_rewards(g):
    transition_probs = {}

    rewards = {}

    for i in range(g.rows):
        for j in range(g.columns):
            s = (i, j)
            if not g.is_terminal(s):
                for a in ACTION_SPACE:
                    s_next = g.get_next_state(s, a)
                    transition_probs[(s, a, s_next)] = 1 # prob of s with action to reach s_next is 1 because deterministic
                    if s_next in g.rewards:
                        rewards[(s, a, s_next)] = g.rewards[s_next]
    return transition_probs, rewards

In [24]:
def evaluate_deterministic_policy(g, policy):
    V = {}
    for s in g.all_states():
        V[s] = 0
    iterations = 0

    while True:
        biggest_change = 0
        for s in g.all_states():
            old_v = V[s]
            new_v = 0
            for a in ACTION_SPACE:
                for s_next in g.all_states():
                    action_prob = 1 if policy.get(s) == a else 0

                    r = rewards.get((s, a, s_next), 0)

                    new_v += action_prob * transition_probs.get((s, a, s_next), 0) * (r + gamma * V[s_next])

            V[s] = new_v
            biggest_change = max(biggest_change, np.abs(old_v - V[s]))

        iterations += 1
        if biggest_change < delta:
            break

    return V

In [25]:
def get_optimal_policy(g, policy):
    while True:
        V = evaluate_deterministic_policy(g, policy)

        is_policy_converged = True
        for s in g.actions.keys():
            old_a = policy[s]
            new_a = None
            best_value = float('-inf')

            for a in ACTION_SPACE:
                v = 0
                for s_next in g.all_states():
                    r = rewards.get((s, a, s_next), 0)
                    v += transition_probs.get((s, a, s_next), 0) * (r + gamma * V[s_next])

                if v > best_value:
                    best_value = v
                    new_a = a
            
            policy[s] = new_a

            if new_a != old_a:
                is_policy_converged = False
            
        if is_policy_converged:
            break
    return V, policy

In [26]:
# initialize variable
g = grid_standard()
delta = 1e-3
gamma = 0.9

# random policy
policy = {}
for s in g.actions.keys():
    policy[s] = np.random.choice(ACTION_SPACE)

print("initial policy")
print_policy(policy, g)
print("")

transition_probs, rewards = t_probs_rewards(g)

V, policy = get_optimal_policy(g, policy)
print("optimal policy")
print_policy(policy, g)
print("V")
print_values(V, g)

initial policy
-------------------------
 U | R | U |   |
-------------------------
 R |   | L |   |
-------------------------
 U | L | D | U |

optimal policy
-------------------------
 R | R | R |   |
-------------------------
 U |   | U |   |
-------------------------
 U | R | U | L |
V
-------------------------
 0.81| 0.90| 1.00| 0.00|
-------------------------
 0.73| 0.00| 0.90| 0.00|
-------------------------
 0.66| 0.73| 0.81| 0.73|
